In [552]:
pip install rake-nltk

Note: you may need to restart the kernel to use updated packages.


In [553]:
import pandas as pd
import numpy as np

from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer #tokenizes a collection of words extracted from a text doc
from ast import literal_eval #This evaluates whether an expresion is a Python datatype or not

In [554]:
data = pd.read_csv('goodreads_data.csv')
print(data.shape)
data.head()

(10000, 8)


,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [555]:
data.isnull().sum()


Unnamed: 0      0
Book            0
Author          0
Description    77
Genres          0
Avg_Rating      0
Num_Ratings     0
URL             0
dtype: int64

In [556]:
data = data.fillna('missing value')


In [557]:
rec_author = 'Harper Lee'
data[data['Author'] == rec_author]

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1432,1432,Go Set a Watchman,Harper Lee,From Harper Lee comes a landmark new novel set...,"['Fiction', 'Historical Fiction', 'Classics', ...",3.31,"262,779",https://www.goodreads.com/book/show/24817626-g...


In [558]:
data = data.rename(columns={'Unnamed: 0': 'index'})

In [559]:
data.columns

Index(['index', 'Book', 'Author', 'Description', 'Genres', 'Avg_Rating',
       'Num_Ratings', 'URL'],
      dtype='object')

In [560]:
data1 = data[['index','Book', 'Author', 'Description', 'Genres']].head(3000)
data1.head()

,index,Book,Author,Description,Genres
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',..."
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical..."
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp..."
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',..."


In [561]:
data1.isnull().sum()


index          0
Book           0
Author         0
Description    0
Genres         0
dtype: int64

In [562]:
data1 = data1.map(lambda x: x.lower() if type(x) == str else x)
data1.head()

,index,Book,Author,Description,Genres
0,0,to kill a mockingbird,harper lee,the unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical fiction', ..."
1,1,harry potter and the philosopher’s stone (harr...,j.k. rowling,harry potter thinks he is an ordinary boy - un...,"['fantasy', 'fiction', 'young adult', 'magic',..."
2,2,pride and prejudice,jane austen,"since its immediate success in 1813, pride and...","['classics', 'fiction', 'romance', 'historical..."
3,3,the diary of a young girl,anne frank,discovered in the attic in which she spent the...,"['classics', 'nonfiction', 'history', 'biograp..."
4,4,animal farm,george orwell,librarian's note: there is an alternate cover ...,"['classics', 'fiction', 'dystopia', 'fantasy',..."


In [563]:
# import ast

In [564]:
# data1['Genres'] = data1['Genres'].astype(str)

# data1['Genres'] = data1['Genres'].str.replace('[', '').str.replace(']', '').str.replace("'", '')


In [565]:
# data1.head()

In [566]:
# data1['Genres'] = data1['Genres'].map(lambda x: x.split(','))
# data1['Genres']

In [567]:
data1['Author'] = data1['Author'].apply(lambda x: [name.replace(" ", "") for name in x.split(',')])

data1['Author'].head()


0       [harperlee]
1     [j.k.rowling]
2      [janeausten]
3       [annefrank]
4    [georgeorwell]
Name: Author, dtype: object

In [568]:
data1.head()

,index,Book,Author,Description,Genres
0,0,to kill a mockingbird,[harperlee],the unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical fiction', ..."
1,1,harry potter and the philosopher’s stone (harr...,[j.k.rowling],harry potter thinks he is an ordinary boy - un...,"['fantasy', 'fiction', 'young adult', 'magic',..."
2,2,pride and prejudice,[janeausten],"since its immediate success in 1813, pride and...","['classics', 'fiction', 'romance', 'historical..."
3,3,the diary of a young girl,[annefrank],discovered in the attic in which she spent the...,"['classics', 'nonfiction', 'history', 'biograp..."
4,4,animal farm,[georgeorwell],librarian's note: there is an alternate cover ...,"['classics', 'fiction', 'dystopia', 'fantasy',..."


In [569]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sugiyamamikuu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sugiyamamikuu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [570]:
data1['keywords'] = ''

# Iterate over each row in the DataFrame
for index, row in data1.iterrows():
    description = row['Description']
    
    r = Rake()
    
    r.extract_keywords_from_text(description)
    
    keyword_dict_scores = r.get_word_degrees()
    
    data1.at[index, 'keywords'] = list(keyword_dict_scores.keys())

# Check the result
data1[['Description', 'keywords']].head()

,Description,keywords
0,the unforgettable novel of a childhood in a sl...,"[unforgettable, novel, childhood, sleepy, sout..."
1,harry potter thinks he is an ordinary boy - un...,"[harry, potter, thinks, ordinary, boy, rescued..."
2,"since its immediate success in 1813, pride and...","[since, immediate, success, 1813, pride, preju..."
3,discovered in the attic in which she spent the...,"[discovered, attic, spent, last, years, life, ..."
4,librarian's note: there is an alternate cover ...,"[librarian, note, alternate, cover, edition, b..."


In [571]:
data1.set_index('Book', inplace = True)
data1.head()

,index,Author,Description,Genres,keywords
Book,,,,,
to kill a mockingbird,0,[harperlee],the unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical fiction', ...","[unforgettable, novel, childhood, sleepy, sout..."
"harry potter and the philosopher’s stone (harry potter, #1)",1,[j.k.rowling],harry potter thinks he is an ordinary boy - un...,"['fantasy', 'fiction', 'young adult', 'magic',...","[harry, potter, thinks, ordinary, boy, rescued..."
pride and prejudice,2,[janeausten],"since its immediate success in 1813, pride and...","['classics', 'fiction', 'romance', 'historical...","[since, immediate, success, 1813, pride, preju..."
the diary of a young girl,3,[annefrank],discovered in the attic in which she spent the...,"['classics', 'nonfiction', 'history', 'biograp...","[discovered, attic, spent, last, years, life, ..."
animal farm,4,[georgeorwell],librarian's note: there is an alternate cover ...,"['classics', 'fiction', 'dystopia', 'fantasy',...","[librarian, note, alternate, cover, edition, b..."


In [572]:
data1 = data[['Book', 'Author', 'Description', 'Genres']]

In [573]:
# Function to create a bag of words (bow) for each row
def create_bow(row):
    words = ''
    for col in row.index:  # Iterate over each column in the row
        value = row[col]
        if isinstance(value, list):
            value = ' '.join(value)  # Join list elements if it's a list
        elif not isinstance(value, str):
            value = str(value)  # Convert non-string values to string

        words += value + ' '  # Concatenate values into a single string
    return words.strip()  # Strip trailing whitespace

# Apply the function to each row to create the 'bow' column
data1['bow'] = data1.apply(create_bow, axis=1)

# Check the result
print(data1[['bow']].head())



                                                 bow
0  To Kill a Mockingbird Harper Lee The unforgett...
1  Harry Potter and the Philosopher’s Stone (Harr...
2  Pride and Prejudice Jane Austen Since its imme...
3  The Diary of a Young Girl Anne Frank Discovere...
4  Animal Farm George Orwell Librarian's note: Th...


/var/folders/4q/vpk3z6s16t1622d9qwvj5l640000gn/T/ipykernel_53295/2007422999.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['bow'] = data1.apply(create_bow, axis=1)


In [574]:
data1.head()

,Book,Author,Description,Genres,bow
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",To Kill a Mockingbird Harper Lee The unforgett...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",Harry Potter and the Philosopher’s Stone (Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",Pride and Prejudice Jane Austen Since its imme...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",The Diary of a Young Girl Anne Frank Discovere...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",Animal Farm George Orwell Librarian's note: Th...


In [588]:
data1.set_index('Book', inplace = True)
data1.head()

,Author,Description,Genres,bow
Book,,,,
To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",To Kill a Mockingbird Harper Lee The unforgett...
"Harry Potter and the Philosopher’s Stone (Harry Potter, #1)",J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",Harry Potter and the Philosopher’s Stone (Harr...
Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",Pride and Prejudice Jane Austen Since its imme...
The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",The Diary of a Young Girl Anne Frank Discovere...
Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",Animal Farm George Orwell Librarian's note: Th...


In [590]:
#instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(data1['bow'])

#create a Series for movie titles so they are associated to an ordered numerical list, we will use this later to match index
indices = pd.Series(data1.index)
print(indices)

0                                   To Kill a Mockingbird
1       Harry Potter and the Philosopher’s Stone (Harr...
2                                     Pride and Prejudice
3                               The Diary of a Young Girl
4                                             Animal Farm
                              ...                        
9995                      Breeders (Breeders Trilogy, #1)
9996                                               Dynamo
9997                                The Republic of Trees
9998                       Waking Up (Healing Hearts, #1)
9999                   Bits and Pieces: Tales and Sonnets
Name: Book, Length: 10000, dtype: object


In [592]:
count_matrix

<10000x73093 sparse matrix of type '<class 'numpy.int64'>'
	with 1119127 stored elements in Compressed Sparse Row format>

In [594]:
type(count_matrix)

scipy.sparse._csr.csr_matrix

In [596]:
c = count_matrix.todense()
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [597]:
#Print count_matrix for 0th row
print(count_matrix[0,:]) #This shows all words and their frequency in bow of 0th row

  (0, 59017)	7
  (0, 32191)	3
  (0, 38236)	3
  (0, 26297)	2
  (0, 33620)	2
  (0, 58348)	4
  (0, 61093)	1
  (0, 40804)	1
  (0, 41342)	4
  (0, 10775)	1
  (0, 28905)	4
  (0, 53891)	1
  (0, 54680)	1
  (0, 59393)	1
  (0, 3227)	9
  (0, 13829)	1
  (0, 12640)	1
  (0, 58338)	1
  (0, 50052)	1
  (0, 30520)	4
  (0, 6287)	1
  (0, 7990)	1
  (0, 3140)	2
  (0, 29741)	1
  (0, 6839)	1
  :	:
  (0, 9049)	1
  (0, 65184)	2
  (0, 2390)	1
  (0, 64486)	1
  (0, 11222)	1
  (0, 61227)	1
  (0, 3797)	1
  (0, 2872)	1
  (0, 12694)	1
  (0, 26909)	1
  (0, 7841)	1
  (0, 6199)	1
  (0, 53472)	1
  (0, 59048)	1
  (0, 30394)	1
  (0, 48436)	1
  (0, 4399)	1
  (0, 36414)	1
  (0, 3008)	1
  (0, 34423)	2
  (0, 11285)	1
  (0, 21828)	2
  (0, 27316)	2
  (0, 51592)	1
  (0, 1905)	1


In [600]:
#Gives vocabulary of all words in 'bow' and their counts
count.vocabulary_

{'to': 59017,
 'kill': 32191,
 'mockingbird': 38236,
 'harper': 26297,
 'lee': 33620,
 'the': 58348,
 'unforgettable': 61093,
 'novel': 40804,
 'of': 41342,
 'childhood': 10775,
 'in': 28905,
 'sleepy': 53891,
 'southern': 54680,
 'town': 59393,
 'and': 3227,
 'crisis': 13829,
 'conscience': 12640,
 'that': 58338,
 'rocked': 50052,
 'it': 30520,
 'became': 6287,
 'both': 7990,
 'an': 3140,
 'instant': 29741,
 'bestseller': 6839,
 'critical': 13852,
 'success': 56342,
 'when': 63907,
 'was': 63462,
 'first': 22129,
 'published': 46691,
 '1960': 493,
 'went': 63791,
 'on': 41613,
 'win': 64215,
 'pulitzer': 46742,
 'prize': 46018,
 '1961': 495,
 'later': 33359,
 'made': 35403,
 'into': 30074,
 'academy': 1412,
 'award': 5272,
 'winning': 64282,
 'film': 21954,
 'also': 2815,
 'classic': 11281,
 'compassionate': 12131,
 'dramatic': 17526,
 'deeply': 15151,
 'moving': 38904,
 'takes': 57433,
 'readers': 47856,
 'roots': 50248,
 'human': 27965,
 'behavior': 6432,
 'innocence': 29585,
 'expe

In [602]:
#generating the cosine similarity matrix

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.38910961, 0.46922119, ..., 0.52440974, 0.34667467,
        0.48561805],
       [0.38910961, 1.        , 0.27792883, ..., 0.33886881, 0.25890569,
        0.33363012],
       [0.46922119, 0.27792883, 1.        , ..., 0.58829084, 0.28785557,
        0.48144585],
       ...,
       [0.52440974, 0.33886881, 0.58829084, ..., 1.        , 0.39711264,
        0.60098637],
       [0.34667467, 0.25890569, 0.28785557, ..., 0.39711264, 1.        ,
        0.361119  ],
       [0.48561805, 0.33363012, 0.48144585, ..., 0.60098637, 0.361119  ,
        1.        ]])

In [620]:
data1.reset_index(inplace=True)

data1.head()

,Book,Author,Description,Genres,bow
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",To Kill a Mockingbird Harper Lee The unforgett...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",Harry Potter and the Philosopher’s Stone (Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",Pride and Prejudice Jane Austen Since its imme...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",The Diary of a Young Girl Anne Frank Discovere...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",Animal Farm George Orwell Librarian's note: Th...


In [631]:
# Create a Series that maps movie titles to their index
indices = pd.Series(data1.index, index=data1['Book']).drop_duplicates()

# Now, you can safely find the index of a title
def recommendations(title, n, cosine_sim):
    recommended_movies = []

    # Check if the title exists in the indices
    if title in indices:
        # Get the index of the movie that matches the title
        idx = indices[title]

        # Find the highest cosine similarity for this title with other titles
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)

        # Get the indices of the top N most similar movies
        top_n_indices = score_series.iloc[1:n+1].index

        # Retrieve the movie titles from these indices
        recommended_movies = data1.iloc[top_n_indices]['Book'].values

        return recommended_movies
    else:
        # If the title is not found, return a message
        return f"Movie titled '{title}' not found."


In [634]:
book = '1984'
n = 10

In [636]:
book

'1984'

In [668]:
recommendations(book, n, cosine_sim)

array(['God Speaks', 'The Mahābhārata', 'The Road',
       'A Day No Pigs Would Die', 'The Moon Is Down',
       'Dreaming the Eagle (Boudica, #1)', 'The Orchardist',
       'The Stone Angel', 'Death on the Installment Plan',
       'The Sympathizer'], dtype=object)

In [658]:
indices

Book
To Kill a Mockingbird                                             0
Harry Potter and the Philosopher’s Stone (Harry Potter, #1)       1
Pride and Prejudice                                               2
The Diary of a Young Girl                                         3
Animal Farm                                                       4
                                                               ... 
Breeders (Breeders Trilogy, #1)                                9995
Dynamo                                                         9996
The Republic of Trees                                          9997
Waking Up (Healing Hearts, #1)                                 9998
Bits and Pieces: Tales and Sonnets                             9999
Length: 10000, dtype: int64

In [678]:
idx = indices[book]
print(idx)

6


In [682]:
pd.Series(cosine_sim[indices[indices == movie].index[0]])

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (2126758260.py, line 1)

In [ ]:
#https://www.kaggle.com/code/sasha18/recommender-systems-based-on-content#Count-Vectorizer